In [7]:
import os, requests
from bs4 import BeautifulSoup
from content.models import News
from PIL import Image
import time
import shutil
import wget
from django.core.files import File
import datetime
import progressbar
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

In [9]:
n = News.objects.get(newsId=364955795)
print(n)

유나 몸매 무슨일이야


In [2]:
def date_ranges(start):
    '''
    returns range of dates from start to today
    '''
    start_date = datetime.datetime.strptime(start, '%Y%m%d').date()
    end_date = datetime.date.today() #datetime.date(2018, 1, 4)
    delta = datetime.timedelta(days=1)
    dates = []

    while start_date <= end_date:
        dates.append(start_date.strftime('%Y%m%d'))
        start_date += delta
    return dates
date_ranges('20220205')
# url = "https://pann.nate.com/talk/ranking/d?stdt=20170205&page=1"

['20220205',
 '20220206',
 '20220207',
 '20220208',
 '20220209',
 '20220210',
 '20220211']

In [3]:
class Parser:
    """
    Class for parshing pages on url
    """

    url = ""
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
        "Accept-Language": "en-US, en;q=0.5",
    }

    def __init__(self, url):
        self.url = url

    def getHtml(self):
        r = requests.get(self.url, headers=self.HEADERS)
        
        return r.text

    def parse(self):
        html = self.getHtml()
        soup = BeautifulSoup(html, "lxml")
        div = soup.find("ul", {"class": "post_wrap"})
        return div
    


    newsId = models.IntegerField()
    tmb = models.ImageField(upload_to="tmbs")
    title = models.CharField(max_length=255)
    excerpt = models.CharField(max_length=255)
    href = models.CharField(max_length=255)
    count = models.CharField(max_length=255)
    rcm = models.CharField(max_length=255)


In [8]:
u = "https://thumb.pann.com/tc_100x100/http://fimg4.pann.com/new/download.jsp?FileID=39627719"
local_filename = wget.download(u, bar=None)
print(local_filename)
shutil.copy(local_filename, f"/home/manhee/Disk/Projects/Upwork/KoreanScraper/KorScrap/data/TST1.jpg")

download (414).jsp


'/home/manhee/Disk/Projects/Upwork/KoreanScraper/KorScrap/data/TST1.jpg'

In [26]:
News.objects.all().delete()

(100, {'content.News': 100})

In [4]:
def get_html(url):
    
    parser = Parser(url)
    html = parser.getHtml()
    soup = BeautifulSoup(html, "lxml")
    div = soup.find("div", class_="posting")
    return div
url = "https://pann.nate.com/talk/364984183"
get_html(url)    
    

<div class="posting">
<table>
<tbody>
<tr>
<td>
<div id="contentArea">
<br/>
나는 테런, 오디션 이렇게 엄청 즐겨 했었음<br/>
요즘 계속 보이길래 추억 돋아서 깔아봤는데<br/>
친구들이랑 스페이스바 열심히 눌러대던거 생각남 ㅠㅠㅠ<br/>
<br/>
<br/>
<p class="mweb_img"><a href="https://fimg5.pann.com/new/download.jsp?FileID=63087768" target="_blank"><img id="" name="image" src="https://fimg5.pann.com/new/download.jsp?FileID=63087768" style="border: none;" swaf:cywrite:file_seq="" swaf:cywrite:info="image|036a1ed4-3fdd-4ebe-908e-1a0496a863d4.jpeg|http://fimg5.pann.com/new/download.jsp?FileID=63087768|3481260|62415445" swaf:cywrite:object_id="63087768"/></a></p><br/>
<br/>
진짜 추억 돋는다 어쩜 좋아<br/>
나는 아이디 잃어버려서 뉴비부터 다시 도전…<br/>
기본, 프리스타일 튜토리얼 모두 깨고<br/>
뉴비서버로 들어갔는데 평일이라 그런지 화력이 아쉽ㅠ<br/>
<br/>
친구들 모아서 한판했는데<br/>
이달소 노래도 있고 나름 업글 많이 됐더라<br/>
<br/>
테런은 아직도 화력 좀 있던데<br/>
오디션도 다시 핫해졌음 좋겠어ㅠㅠ<br/>
다들 오디션 같이 하자<br/>
방 수가 적어서 넘 아쉬워ㅠㅠㅠㅠ									</div>
</td>
</tr>
</tbody>
</table>
</div>

In [15]:
# Parser for the blocks

def download(url, imgId):
    local_filename = wget.download(url, bar=None)
    shutil.copy(local_filename, f"/home/manhee/Disk/Projects/Upwork/KoreanScraper/KorScrap/data/{imgId}.jpg")
    return f"/home/manhee/Disk/Projects/Upwork/KoreanScraper/KorScrap/data/{imgId}.jpg"

ranges = date_ranges('20220209')
k = 0
with progressbar.ProgressBar(max_value=(len(ranges) - 1)*50) as bar:
    for j, date_range in enumerate(ranges):
        url = f"https://pann.nate.com/talk/ranking/d?stdt={date_range}&page=1"
        parser = Parser(url)
        ul = parser.parse()
        for i, li in enumerate(ul.findAll('li')):
            img = None
            thumb = li.find('div', class_="thumb")
            title = li.find('dt').find('a').getText()

            link = li.find('a')['href']
            postId = link.split('/')[-1]
            count = li.find('span', class_="count").getText()
            rcm = li.find('span', class_="rcm").getText()
            postDate = datetime.datetime.strptime(date_range, '%Y%m%d')
            #txt
            try:
                txt = li.find('dd', class_="txt").find('a').getText()
            except:
                pass
                
            try:
                img = thumb.find('img')['src']
                local_filename = download(img, postId)
                with open(local_filename, 'rb') as ff:
                    obj.tmb.save(f"{postId}.jpg", File(ff), save=True)
            except Exception as e:
                pass
            html = get_html(f"https://pann.nate.com/talk/{postId}")
            
            try:            
                obj = News.objects.get(newsId=postId)
                
                obj.title= title
                obj.excerpt=txt
                obj.href=link
                obj.count=count
                obj.rcm=rcm
                obj.postDate=postDate
                obj.html=html
                obj.save()
                
            except News.DoesNotExist:
                obj = News(
                    newsId=postId,
                    title=title,
                    excerpt=txt,
                    href=link,
                    count=count,
                    rcm=rcm,
                    postDate=postDate,
                    html=html
                )
                obj.save()

            
            bar.update(k)
            k += 1
        



TypeError: 'NoneType' object is not callable

['20220205',
 '20220206',
 '20220207',
 '20220208',
 '20220209',
 '20220210',
 '20220211']

In [18]:
print(datetime.datetime.strptime('20170205', '%Y%m%d'))

2017-02-05 00:00:00
